## WEC Analysis - Experiment 2

In [3]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import altair as alt
import math
import csv

In [4]:
chart_to_cor = {"Chart 1": "0.8", "Chart 2": "0.6", "Chart 3": "0.4", "Chart 4": "0.2", "Chart 5": "0.0"
                , "Chart 6": "-0.2", "Chart 7": "-0.4", "Chart 8": "-0.6", "Chart 9": "-0.8"}
col_to_cor = {8: "0.8", 7: "0.6", 6: "0.4", 5: "0.2", 4: "0.0"
                , 3: "-0.2", 2: "-0.4", 1: "-0.6", 0: "-0.8"}

In [5]:
raw = pd.read_excel("weca-120.xlsx")

In [6]:
raw.describe()

,ResponseId,TestRightWrong,DirPosCorr,DirNegCorr,MagStrongCorr,MagModCorr,MagWeakCorr,DisClearRel,DisNoClear,RegTightRel,RegLooseRel,BehUpwardTrend,BehDownwardTrend,SpValuesHigh,SpValuesLow,InfVarHigh,InfVarLow
count,120,120,34,36,40,33,36,32,35,36,33,41,39,37,41,41,36
unique,120,3,19,15,11,23,27,12,21,13,21,12,10,15,14,19,14
top,R_3HZ99X3BBvaYFZA,right,"Chart 1,Chart 2","Chart 8,Chart 9","Chart 1,Chart 9","Chart 2,Chart 8","Chart 4,Chart 5,Chart 6","Chart 1,Chart 2,Chart 8,Chart 9","Chart 4,Chart 7,Chart 5,Chart 6","Chart 1,Chart 9","Chart 2,Chart 8","Chart 1,Chart 2,Chart 3","Chart 8,Chart 9","Chart 1,Chart 2","Chart 8,Chart 9","Chart 1,Chart 2","Chart 8,Chart 9"
freq,1,89,13,11,13,4,4,8,5,15,4,12,13,12,12,11,12


In [7]:
#Melt dataset by statements
data = pd.melt(raw,id_vars=["ResponseId","TestRightWrong"], var_name='Statements', value_name='Charts').dropna(axis=0)

In [8]:
data['Charts'] = data['Charts'].apply(lambda x: x.split(',')) #.apply(lambda x: [chart_to_cor[z] for z in x])

In [9]:
#Breakdown the answers and then map them to the corresponding correlations
chartAnswers = data['Charts'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
df = pd.merge(data[['ResponseId','Statements','TestRightWrong']],chartAnswers,left_index=True,right_index=True).rename(columns={"value": "Answer"})
df['Answer'] = df['Answer'].apply(lambda x: chart_to_cor[x])

In [10]:
#Aggregate answers by statements 
count_init = {"-0.8": 0,"-0.6": 0,"-0.4": 0,"-0.2": 0,"0.0": 0,"0.2": 0,"0.4": 0,"0.6": 0,"0.8": 0, }
c = Counter(count_init)
st_counts = []
for name,group in df.groupby("Statements"):
    #print(name)
    #print(group[:5])
    c = Counter(count_init)
    c.update(group["Answer"])
    st_counts.append({"Statement": name, "Values": list(c.items())})
    #print(sorted(c.items()))
#print(st_counts)
d = pd.DataFrame(st_counts)
display(d)

,Statement,Values
0,BehDownwardTrend,"[(-0.8, 33), (-0.6, 28), (-0.4, 14), (-0.2, 2)..."
1,BehUpwardTrend,"[(-0.8, 3), (-0.6, 2), (-0.4, 0), (-0.2, 4), (..."
2,DirNegCorr,"[(-0.8, 28), (-0.6, 21), (-0.4, 12), (-0.2, 9)..."
3,DirPosCorr,"[(-0.8, 6), (-0.6, 5), (-0.4, 1), (-0.2, 6), (..."
4,DisClearRel,"[(-0.8, 23), (-0.6, 18), (-0.4, 5), (-0.2, 1),..."
5,DisNoClear,"[(-0.8, 2), (-0.6, 5), (-0.4, 20), (-0.2, 27),..."
6,InfVarHigh,"[(-0.8, 11), (-0.6, 5), (-0.4, 3), (-0.2, 7), ..."
7,InfVarLow,"[(-0.8, 27), (-0.6, 25), (-0.4, 11), (-0.2, 3)..."
8,MagModCorr,"[(-0.8, 8), (-0.6, 16), (-0.4, 4), (-0.2, 5), ..."
9,MagStrongCorr,"[(-0.8, 31), (-0.6, 10), (-0.4, 0), (-0.2, 1),..."


In [11]:
#Separate correlation and statements into rows
d2 = pd.merge(d['Statement'],pd.DataFrame(d['Values'].apply(lambda x: [z[1] for z in x]).tolist(), index=d.index),
         left_index=True,right_index=True)
d2.rename(columns=col_to_cor,inplace=True)
d2 = pd.melt(d2,id_vars=["Statement"],var_name="Correlation",value_name="Count")
d2['Correlation'] = d2['Correlation'].apply(lambda x: float(x))


In [12]:
#Plot normalized signatures of statements

#dvis = d2[d2["Statement"].isin(['MagStrongCorr','MagModCorr','MagWeakCorr'])]
#display(dvis)
#dvis['sorting'] = pd.Categorical(dvis["Statement"],mag_order)
#dvis.sort_values(['sorting'])
#print(dvis)
c = d2.groupby(['Statement','Correlation'])['Count'].sum()
dmod = pd.DataFrame(c / c.groupby(level=0).sum()).reset_index()

alt.Chart(dmod).mark_area(color='#a00000').encode(
    x='Correlation:N',
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.4))),
).properties(width=150,height=50).facet(row='Statement:O').configure_axis()

alt.FacetChart(...)

**KEY BLOCK FOR CHANGING OBJECT OF ANALYSIS:**

In [113]:
behdir = ['BehDownwardTrend','BehUpwardTrend']
magstatements  = ["MagStrongCorr","MagModCorr","MagWeakCorr"]
reldir = ["DirNegCorr","DirPosCorr"]
disrel = ["DisClearRel","DisNoClear"]
relreg = ["RegLooseRel","RegTightRel"]
spacepos = ["SpValuesLow","SpValuesHigh"]
relpos = ["InfVarLow","InfVarHigh"]

analysing = relpos

dvis = d2[d2["Statement"].isin(analysing)]

c = dvis.groupby(['Statement','Correlation'])['Count'].sum()
dmod = pd.DataFrame(c / c.groupby(level=0).sum()).reset_index()
#p,q,r = list(dmod.groupby('Statement')['Count'].apply(list))
e2chart = alt.Chart(dmod).mark_bar(tooltip=True,color='#a00000').encode(
    x=alt.X('Correlation:O',title=None),
    #y='Count:Q'
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
    #column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Statement:O',title=None,sort=analysing,spacing=15)
).properties(width=100,height=80
).resolve_scale(
    x='independent',
    y='independent'
)

In [114]:
e2chart

alt.Chart(...)

### Comparing results of E1 and E2

The following blocks load and display the results from the first experiment. This requires the last blocks of the wec-analysis notebook to be run.. Further down, we define the Hellinger distance. To calculate the distances, we need to extract the values from the two dataframes from E1 and E2



In [18]:
cat_statements = pd.read_pickle("WECdf-categorised.pkl")

In [19]:
%store -r cat_dict

In [20]:
def process_pair(e1df,concept_,trait_):
    def filter_series_by_category(sentence,list_cat,cat_filter,func):
    #    lst= []
        for word in sentence:
            if (word[0] in list_cat) and (word[1].startswith(func)):
                if list_cat[word[0]] == cat_filter:
                    return word[0]

        #return sorted(list(c.items()),key=lambda x: -x[1])

    list_trait = list(set(e1df['lemmatized'].apply(lambda x: filter_series_by_category(x,cat_dict,trait_,'JJ')).dropna().values))
    list_concept = list(set(e1df['lemmatized'].apply(lambda x: filter_series_by_category(x,cat_dict,concept_,'NN')).dropna().values))

    def cf(x,list_cat,func):
        for word in x:
            if (word[0] in list_cat): #and (word[1].startswith(func)):
                return True
        return False

    trait_filtered = e1df.loc[e1df.lemmatized.apply(lambda x: cf(x,list_trait,'JJ'))].dropna(axis=1)[['ResponseId','Question','Answer','lemmatized','lemma_cat_tagged']]
    concept_filtered = e1df.loc[e1df.lemmatized.apply(lambda x: cf(x,list_concept,'NN'))].dropna(axis=1)[['ResponseId','Question','Answer','lemmatized','lemma_cat_tagged']]
   
    trait_filtered.lemmatized = trait_filtered.lemmatized.apply(tuple)
    concept_filtered.lemmatized = concept_filtered.lemmatized.apply(tuple)

    trait_filtered.lemma_cat_tagged = trait_filtered.lemma_cat_tagged.apply(tuple)
    concept_filtered.lemma_cat_tagged = concept_filtered.lemma_cat_tagged.apply(tuple)

    merged = concept_filtered.merge(trait_filtered,how='inner')
    merged.lemmatized = merged.lemmatized.apply(list)
    merged.lemma_cat_tagged = merged.lemma_cat_tagged.apply(list)
    
#    display(list(set(merged['lemmatized'].apply(lambda x: filter_series_by_category(x,cat_dict,trait_,'JJ')).dropna().values)))
    
    return merged,list_trait,list_concept

To change the concepts and traits to compare, modify the second and third parameters below:

**KEY BLOCK FOR CHANGING OBJECT OF ANALYSIS:**

In [90]:
merged,traits,concepts = process_pair(cat_statements,'RELATION','MAGNITUDE')

Below we need to specify the file containing the synonym mapping (e.g. map every discernibility word to "clear" for the comparison). Alternative arrangements can be used for this, but this supplementary material is based on loading each file at a time.

In [91]:
compare_scale = {}
with open('magscale-filtered.csv',encoding='utf-8-sig') as f:
#     magscale = dict(filter(None, csv.reader(f)))
    #compare_scale = {k: int(v) for k,v in csv.reader(f)}
    compare_scale = {k: v for k,v in csv.reader(f)}

In [23]:
def assign_multi_value(sentence,scale):
    svalues = {}
    for word in sentence:
        if word[0] in scale:
            svalues[word[0]] = scale[word[0]]
    return svalues

def tag_values(data,scale):
    data['multiscale'] =  data['lemmatized'].apply(lambda x: assign_multi_value(x,scale))
    ms = data['multiscale'].apply(pd.Series).reset_index() #.dropna(how='all',subset=scale.keys())
    #print(ms.dropna(how='all',subset=scale.keys()))
    #print(ms.dropna(how='all',subset=scale.keys()).melt(id_vars='index')[['index', 'value']].set_index('index'))
    #print(pd.merge(merged[['ResponseId','Question','lemma_cat_tagged']],ms.dropna(how='all',subset=scale.keys()).melt(id_vars='index')[['index', 'value']].set_index('index'),left_index=True,right_index=True).dropna())
    ms = ms.melt(id_vars='index')[['index', 'value']].set_index('index')
    merg =  pd.merge(merged[['ResponseId','Question','lemma_cat_tagged']],ms,left_index=True,right_index=True).dropna()
    #print(len(merg))
    return merg.drop('ResponseId',axis=1).reset_index(drop=True)

#%store -r e1df
def group_answers(to_group):
    special = ['NEGATE']
    concepts = ['RELATION','BEHAVIOUR','SPACE','INFERENCES','GRAPHIC','OTHER']
    traits = ['MAGNITUDE','DIRECTION','POSITION','DISCERNIBILITY','REGULARITY']
    floatQ = {'C8P': 0.8,'C6P': 0.6,'C4P':0.4,'C2P':0.2,'C0': 0,'C2N': -0.2,'C4N': -0.4,'C6N':-0.6,'C8N': -0.8}

    to_group['special'] = to_group['lemma_cat_tagged'].apply(lambda x: True if len([s for s in x if s in special]) > 0 else False)
    sp = to_group['special'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
    data = pd.merge(to_group,sp,left_index=True,right_index=True).rename(columns={'value_x': 'Cat', 'special': 'Negative'})
    #print(data)
    #print(data.groupby(['Question','Cat','Negative']).apply(len))
    group_e1df = data.groupby(['Question','Cat','Negative']).apply(len).reset_index().rename(columns={0: "Count"})
#    print(group_e1df.sort_values(by=['Cat','Question']))
    group_e1df['Correlation'] = group_e1df['Question'].apply(lambda x: floatQ[x])
    return group_e1df

In [92]:
ed1fc = tag_values(merged,compare_scale)
grouped_data = group_answers(ed1fc)

In [93]:
grouped_data['Cat'] = grouped_data['Cat'].apply(lambda x: {1 :'Weak',2: 'Moderate', 3:'Strong'}[float(x)])

In [94]:
exp1 = alt.Chart(grouped_data[grouped_data['Negative'] == False]).mark_bar().encode( #[group_e1df['Negative'] == False]).mark_bar().encode(
    x='Correlation:O',
    y=alt.Y('Count:Q',title=None),
    column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O')).properties(width=100,height=80) #,title='Occurrence of {} words per correlation'.format(trait))
exp1

alt.Chart(...)

Normalize and plot values.  
The first function is used when we have negative values. For some combinations, we won't have that.

In [79]:
def normalize_groups_neg(data):
    c = data.groupby(['Cat','Negative','Correlation'])['Count'].sum()
    wec1df = pd.DataFrame(c / c.groupby(level=0).sum()).reset_index()
    return wec1df

In [95]:
def normalize_groups(data):
    #print(data)
    c = data.groupby(['Cat','Correlation'])['Count'].sum()
    #print(c)
    wec1df = pd.DataFrame(c / c.groupby(level=0).sum()).reset_index()
    #print(wec1df)
    return wec1df

In [81]:
e1_normalized = normalize_groups_neg(grouped_data)
#e1_normalized = normalize_groups(grouped_data[grouped_data['Negative'] == False])
#x,y,w,z = e1_normalized.groupby(['Cat','Negative'])['Count'].apply(list)
#x,y = grouped_data[grouped_data['Negative'] == False].groupby(['Cat','Negative'])['Count'].apply(list)
#print(x)
#x.insert(4,0)
#x.insert(5,0)
#y.insert(4,0)
#x,y = wec1df[wec1df['Negative'] == False].groupby(['Cat','Negative'])['Count'].apply(list)
#x,y = wec1df[wec1df['Cat'] == 'clear'].groupby(['Cat','Negative'])['Count'].apply(list)
#wec1df[wec1df['Negative'] == False]
binary1,binary1_neg,binary2,binary2_neg = grouped_data.groupby(['Cat','Negative'],as_index=False)

ValueError: too many values to unpack (expected 4)

Code below only for Magnitude:

In [96]:
e1_normalized = normalize_groups(grouped_data[grouped_data['Negative'] == False])
x,y,z = e1_normalized.groupby('Cat')['Count'].apply(list)

In [97]:
e1chart = alt.Chart(e1_normalized).mark_bar().encode(
    x=alt.X('Correlation:N',title=None),
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
   # column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"],spacing=15)).properties(width=100,height=80).resolve_axis(x='independent') #,title='Occurrence of {} words per correlation'.format(trait))
e1chart

alt.Chart(...)

The code below is used to fill in the values that are missing from E1, to enable the calculation of the Hellinger distance. It would fail with missing values otherwise.

In [84]:
x[1]['Count'] = x[1]['Count'].divide(x[1]['Count'].sum())
#x[1].concat(pd.DataFrame([{'Correlation': 0.2, 'Count': 0},{'Correlation': -0.2, 'Count': 0}]))
x[1].set_index('Question',inplace=True)
x[1].loc['C4N'] = ['clear','False',0,-0.4]
x[1].loc['C2N'] = ['clear','False',0,-0.2]
x[1].loc['C2P'] = ['clear','False',0,0.2]
x[1].sort_values(by='Correlation',inplace=True)
#x[1].reset_index()
#display(x[1].sort_values(by='Correlation'))
e1chart = alt.Chart(x[1]).mark_bar().encode(
    x='Correlation:O',
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.4))),
    #column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='')).properties(width=100,height=100) #,title='Occurrence of {} words per correlation'.format(trait))
    #row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"])).properties(width=100,height=100) #,title='Occurrence of {} words per correlation'.format(trait))
e1chart

TypeError: 'float' object is not subscriptable

## Calculating distances between histograms

Function h3 below is taken from https://gist.github.com/larsmans/3116927:

In [57]:
from scipy.linalg import norm
def h3(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) /  np.sqrt(2)

In [89]:
#TSNE
#y.insert(4,0)
#x.insert(8,0)
#x,y,z = e1_normalized.groupby('Cat')['Count'].apply(list)
print(h3(p,x),h3(q,y),h3(r,z))

alt.Chart(e1_normalized).mark_bar().encode(
    x='Correlation:O',
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
   # column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"],spacing=15)).properties(width=100,height=80) #,title='Occurrence of {} words per correlation'.format(trait))

ValueError: operands could not be broadcast together with shapes (9,) (7,) 

### Semantic grouping

For certain traits, we want to do a step-by-step comparison of magnitude, showing the differences when we move words between groups.

In [86]:
#SLIGHT = 2
compare_scale['slight'] = 2
compare_scale['few'] = 1
ed1fc = tag_values(merged,compare_scale)
grouped_data = group_answers(ed1fc)
grouped_data['Cat'] = grouped_data['Cat'].apply(lambda x: {1 :'Weak',2: 'Moderate', 3:'Strong'}[float(x)])
e1_normalized = normalize_groups(grouped_data[grouped_data['Negative'] == False])
x,y,z = e1_normalized.groupby('Cat')['Count'].apply(list)
#y.insert(4,0)
#x.insert(8,0)
x.insert(4,0)
print(h3(p,x),h3(q,y),h3(r,z))
alt.Chart(e1_normalized).mark_bar().encode(
    x=alt.X('Correlation:O',title=None),
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
   # column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"])).properties(width=100,height=100).resolve_axis(x='independent') #,title='Occurrence of {} words per correlation'.format(trait))

ValueError: operands could not be broadcast together with shapes (9,) (10,) 

In [41]:
#MINMIMAL = 1
compare_scale['condensed'] = 2
compare_scale['concentrated'] = 2
compare_scale['broad'] = 2
ed1fc = tag_values(merged,compare_scale)
grouped_data = group_answers(ed1fc)
grouped_data['Cat'] = grouped_data['Cat'].apply(lambda x: {1 :'Weak',2: 'Moderate', 3:'Strong'}[float(x)])
e1_normalized = normalize_groups(grouped_data[grouped_data['Negative'] == False])
x,y,z = e1_normalized.groupby('Cat')['Count'].apply(list)
y.insert(4,0)
#x.insert(4,0)
print(h3(p,x),h3(q,y),h3(r,z))
alt.Chart(e1_normalized).mark_bar().encode(
    x=alt.X('Correlation:O',title=None),
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
   # column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"])).properties(width=100,height=100).resolve_axis(x='independent') #,title='Occurrence of {} words per correlation'.format(trait))

0.2547232350286169 0.23206244266374607 0.09683277711063616


alt.Chart(...)

In [54]:
#MANY removed
#compare_scale.pop('large')
#compare_scale.pop('packed')
#compare_scale.pop('more')
ed1fc = tag_values(merged,compare_scale)
grouped_data = group_answers(ed1fc)
grouped_data['Cat'] = grouped_data['Cat'].apply(lambda x: {1 :'Weak',2: 'Moderate', 3:'Strong'}[float(x)])
e1_normalized = normalize_groups(grouped_data[grouped_data['Negative'] == False])
x,y,z = e1_normalized.groupby('Cat')['Count'].apply(list)
#x.insert(4,0)
y.insert(4,0)
y.insert(2,0)
#z.insert(0,0)

#z.insert(4,0)
print(h3(p,x),h3(q,y),h3(r,z))
alt.Chart(e1_normalized).mark_bar().encode(
    x=alt.X('Correlation:O',title=None),
    y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5))),
   # column=alt.Column('Negative:N',title='Negative statement?'),
    row=alt.Row('Cat:O',title='',sort=["Strong","Moderate","Weak"],spacing=15)
).properties(width=100,height=80).resolve_axis(x='independent') #,title='Occurrence of {} words per correlation'.format(trait))


0.2547232350286169 0.11663767518956535 0.09683277711063616


alt.Chart(...)

## Cross-calculating and comparing

For binary traits, we need to combine the affirmative counts with the negative for the opposite value.  

E.g. for clear and vague, we need to sum clear with *not vague* and vague with *not clear*.  

The calculation is done using variables *binary1, binary1_neg, binary2, binary2_neg* as defined above when plotting the normalized counts.

In [232]:
def combine_traits(x,z,list_corr):

    x1 = {float(a): float(0) for a in list_corr}
    #for a in x[1]:
    #    print(x[1][a])
    for row in x[1][['Correlation','Count']].values:
        val = row[1]
        x1[row[0]] += val
    for row in z[1][['Correlation','Count']].values:
        val = row[1]
        x1[row[0]] += val
    dz = pd.DataFrame.from_dict(data=x1,orient='index').reset_index().rename(columns={'index': 'Correlation',0: 'Count'}) #,columns=['Correlation','Count'])
    #display(dz)
    return dz

In [272]:
group_1_and_2neg = combine_traits(binary1,binary2_neg,count_init)
c = group_1_and_2neg.groupby(['Correlation'])['Count'].sum()
first_group = pd.DataFrame(c / c.sum()).reset_index()
first_chart = alt.Chart(first_group).mark_bar().encode(
x=alt.X('Correlation:O',title=None),
y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5)))).properties(width=100,height=80)

In [273]:
group_2_and_1neg = combine_traits(binary2,binary1_neg,count_init)
c = group_2_and_1neg.groupby(['Correlation'])['Count'].sum()
second_group = pd.DataFrame(c / c.sum()).reset_index()
second_chart = alt.Chart(second_group).mark_bar().encode(
x=alt.X('Correlation:O',title=None),
y=alt.Y('Count:Q',title=None,scale=alt.Scale(domain=(0, 0.5)))).properties(width=100,height=80)

In [274]:
(first_chart & second_chart)

alt.VConcatChart(...)

In [199]:
second_chart

alt.Chart(...)

Combining charts from the first and second experiment

In [275]:
((first_chart & second_chart) | e2chart).configure_concat(
    spacing=15
)

alt.HConcatChart(...)

### Formatting

This should be the first lines run

In [28]:
def wec_theme():
    font = "Avenir"
    
    return {
        "config" : {
             "mark": {'color': '#000050'},
             "title": {'font': font,  "anchor": "end"},
             "axis": {
                  "labelFont": font,
                  "titleFont": font
             },
             "headerColumn": {
                   "labels": False
             },
             "header": {
                  "labelFont": font,
                  'labelFontSize': 12,
                  "titleFont": font, 
             },
             "legend": {
                  "labelFont": font,
                  "titleFont": font
             }
        }
    }
alt.themes.register('wec_theme',wec_theme)
alt.themes.enable('wec_theme')

ThemeRegistry.enable('wec_theme')